## CONFIG

In [ ]:
MISSING_VAL_COLUMNS = ['car']
MISSING_VAL_COLUMNS_TO_FILL = ['Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50']

DROP_COL_LIST_WITH_ONE_CLASS = ['toCoupon_GEQ5min']
TEN_BEST_FEATURES_WITH_MUTUAL_INFO_CLASSIF = ['destination','passanger','weather','time','coupon','expiration',
                                            'has_children','income','CoffeeHouse','direction_same']
TEN_BEST_FEATURES_WITH_LIN_REG_BKWD = ['destination','weather','coupon','expiration','gender','maritalStatus',
                                       'occupation','income','CoffeeHouse','direction_opp']
TEN_BEST_FEATURES_WITH_LIN_REG_FWD =['destination','passanger','time','coupon','expiration','occupation','Bar',
                                     'CoffeeHouse','Restaurant20To50','direction_same'] 
TEN_BEST_FEATURES_WITH_LOG_REG_BKWD = ['destination','passanger','weather','coupon','expiration','gender','occupation',
                                      'income','CoffeeHouse','direction_same']
TEN_BEST_FEATURES_WITH_LOG_REG_FWD =['destination','passanger','weather','coupon','expiration','gender',
                                     'occupation','CoffeeHouse','toCoupon_GEQ15min','direction_same']
TEN_BEST_FEATURES_OBSERVED_SELECTION = ['passanger', 'coupon','CoffeeHouse', 'destination', 'expiration', 'toCoupon_GEQ25min',
                                        'Bar', 'gender', 'Restaurant20To50','temperature' ]

In [2]:
Categorical_Features = ['destination', 'passanger', 'weather', 'time', 'coupon', 'expiration', 'gender', 'age', 'maritalStatus',
                        'education', 'occupation', 'income', 'Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20',
                        'Restaurant20To50']
Numerical_Features = ['temperature', 'has_children', 'toCoupon_GEQ5min', 'toCoupon_GEQ15min', 'toCoupon_GEQ25min',
                      'direction_same', 'direction_opp', 'Y']

In [4]:
pip install imbalanced-learn==0.7

  Using cached imbalanced_learn-0.7.0-py3-none-any.whl (167 kB)
Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [87]:
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
import lightgbm as lgbm
from sklearn.model_selection import cross_val_score
import pickle
from joblib import dump, load

In [7]:
current_path = os.getcwd()
dirname, filename = os.path.split(current_path)
print(dirname, filename)

C:\Users\saksh\Documents\20220502_Mlflow_Example notebooks


In [8]:
data_dir= os.path.join(dirname, 'files')
print(data_dir)

C:\Users\saksh\Documents\20220502_Mlflow_Example\files


In [9]:
train_fengg_file_path = os.path.join(data_dir, "step3\\traindf_with_feature_engg.parquet")
traindf_with_feature_engg= pd.read_parquet(train_fengg_file_path)

In [10]:
traindf_with_feature_engg.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,2,1,2,80,2,1,1,1,2,2,...,3,1,3,1,1,1,0,0,1,0
1,2,2,2,80,2,1,2,2,1,1,...,2,2,3,3,1,0,0,0,1,1
2,1,3,1,30,1,2,2,1,1,1,...,3,1,3,2,1,1,0,1,0,1
3,2,1,2,80,2,2,1,1,2,2,...,3,2,4,5,1,1,0,0,1,1
4,1,3,2,80,2,2,2,1,1,1,...,3,1,3,2,1,0,0,1,0,1


In [11]:
traindf_with_feature_engg.drop(DROP_COL_LIST_WITH_ONE_CLASS, axis=1, inplace=True)

## Analysis Functions

## Action Functions

In [12]:
def test_train_split(df, frac_train):
    frac= frac_train
    train_df = df.sample(frac=frac)
    test_df = df.drop(train_df.index)
    return train_df, test_df 

In [13]:
def split_into_XnY(df, dep_col):
    Ytrain = df[dep_col]
    Xtrain = df.drop([dep_col], axis=1)
    return Xtrain, Ytrain

In [14]:
def statsmodel_logit(xtrain, ytrain, xvalidation, yvalidation):
    logit_model=sm.Logit(ytrain,xtrain).fit()
    print(logit_model.summary2())
    y_pred = logit_model.predict(xvalidation)
    prediction = list(map(round, y_pred))
    print(accuracy_score(yvalidation, prediction))
    return logit_model

In [15]:
def model_logit_sklearn(xtrain, ytrain, xvalidation, yvalidation):
    logreg = LogisticRegression().fit(xtrain, ytrain)
    y_pred = logreg.predict(xvalidation)
    print(accuracy_score(yvalidation, y_pred))
    return logreg

In [6]:
def balancing_data_with_SMOTE(xtrain, ytrain):
    # if the dataset is unbalanced i.e no. of values in each level of target variable varies a lot.Then it should be balanced. 
    # For this SMOTE is used.
    sm = SMOTE(random_state=42)
    X_train_sm, y_train_sm= sm.fit_resample(xtrain, ytrain)
    print(y_train_sm.value_counts())
    return X_train_sm, y_train_sm

In [28]:
def GridSearchCV_forBestParams(xtrain, ytrain):
    #Input Xtrain and Ytrain should be balanced
    gbm1 = lgbm.LGBMClassifier({
        'early_stopping_rounds':100, 'bagging_fraction': 0.5, 'bagging_freq' :5,  'objective':'binary',   
        'num_boost_round':1000,'num_threads':4,
    })
    gridParams = {
        'boosting_type':['dart','gbdt'],
        'metric':[
            'binary_logloss',
            'binary_error',
            'auc'
        ],
        'num_leaves': np.array([8,16,32]),
        'learning_rate': [0.05,0.1,0.15, 0.2, 0.25] ,
        'max_depth' : [2,3,4,5] 
    }
    grid = GridSearchCV(gbm1, gridParams, verbose=2, cv=4, n_jobs=1)
    grid.fit(xtrain, ytrain)
    best_params = grid.best_params_
    return best_params

In [90]:
def lgbm_model(xtrain, ytrain,xvalidation, yvalidation ):
    #Using LGBM Classifier with best parameters and then training the training data
    #Input Xtrain and Ytrain should be balanced
    grid_params = GridSearchCV_forBestParams(xtrain, ytrain)
    parameters = {'objective':'binary',  'num_boost_round':1000 ,
        'num_threads':4,
    }
    parameters.update(grid_params)
#     param_str = str(parameters)
#     param_str =param_str.replace(':','=')
#     param_str = param_str.replace('{', '')
#     param_str = param_str.replace('}', '')
    gbm = lgbm.LGBMClassifier(**parameters)
    gbm.fit(xtrain, ytrain,
            early_stopping_rounds= 100, 
            eval_set=[(xtrain, ytrain), (xvalidation, yvalidation)])
    y_pred = gbm.predict(xvalidation)
    y_pred_prob = gbm.predict_proba(xvalidation)
    print('Accuracy for Train Set', cross_val_score(gbm, xtrain, ytrain, scoring='accuracy', cv=4, n_jobs=4))
    print('Accuracy for Validation Set', cross_val_score(gbm, xvalidation, yvalidation, scoring='accuracy', cv=4, n_jobs=4))
    return gbm
    

## Flow

In [16]:
fraction_train = 0.7
traindf, validationdf = test_train_split(traindf_with_feature_engg, fraction_train)

In [17]:
Xtrain, Ytrain = split_into_XnY(traindf, 'Y')

In [18]:
Xval, Yval = split_into_XnY(validationdf, 'Y')

In [19]:
Xtrain_tenfeat = Xtrain[TEN_BEST_FEATURES_OBSERVED_SELECTION]

In [20]:
Xval_tenfeat = Xval[TEN_BEST_FEATURES_OBSERVED_SELECTION]

In [21]:
statsmodel_logit(Xtrain_tenfeat, Ytrain, Xval_tenfeat, Yval)

Optimization terminated successfully.
         Current function value: 0.646361
         Iterations 5
                          Results: Logit
Model:               Logit            Pseudo R-squared: 0.055     
Dependent Variable:  Y                AIC:              8054.2624 
Date:                2022-05-09 13:42 BIC:              8121.6096 
No. Observations:    6215             Log-Likelihood:   -4017.1   
Df Model:            9                LL-Null:          -4252.2   
Df Residuals:        6205             LLR p-value:      1.4339e-95
Converged:           1.0000           Scale:            1.0000    
No. Iterations:      5.0000                                       
------------------------------------------------------------------
                   Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
------------------------------------------------------------------
passanger         -0.4772   0.0338 -14.1211 0.0000 -0.5434 -0.4110
coupon             0.5990   0.0543  11.0377 0.0000  0

In [85]:
model = model_logit_sklearn(Xtrain_tenfeat, Ytrain, Xval_tenfeat, Yval)


0.6441441441441441


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [88]:
model_file_path = os.path.join(data_dir, "step5\\model.joblib")
dump(model, model_file_path)

['C:\\Users\\saksh\\Documents\\20220502_Mlflow_Example\\files\\step5\\model.joblib']

In [24]:
xtrain_balanced, ytrain_balanced = balancing_data_with_SMOTE(Xtrain_tenfeat, Ytrain)

0    3523
1    3523
Name: Y, dtype: int64


In [29]:
GridSearchCV_forBestParams(xtrain_balanced, ytrain_balanced)

Fitting 4 folds for each of 360 candidates, totalling 1440 fits
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=8; total time=   0.2s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=16; total t

[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=16; total time=   0.1s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=16; total time=   0.1s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=16; total time=   0.1s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, ma

[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth

[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_logloss, num_leaves=32; total time=   0.1s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_error, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4

[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_error, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_error, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_d

...

[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=2, metric=binary_error, num_leaves=32; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=2, metric=binary_error, num_leaves=32; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=2, metric=binary_error, num_leaves=32; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=2, metric=binary_error, num_leaves=32; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=2, metric=auc, num_leaves=8; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=2, metric=auc, num_leaves=8; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=2, metric=auc, num_leaves=8; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=2, metric=auc, num_leaves=8; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=2, metric=auc, num_leaves=16; total time=   0.0s
[CV] 

[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=4, metric=binary_error, num_leaves=32; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=4, metric=auc, num_leaves=8; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=4, metric=auc, num_leaves=8; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=4, metric=auc, num_leaves=8; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=4, metric=auc, num_leaves=8; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=4, metric=auc, num_leaves=16; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=4, metric=auc, num_leaves=16; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=4, metric=auc, num_leaves=16; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.2, max_depth=4, metric=auc, num_leaves=16; total time=   0.0s
[CV] END boosting_type=gbdt, lea

[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=2, metric=auc, num_leaves=8; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=2, metric=auc, num_leaves=8; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=2, metric=auc, num_leaves=16; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=2, metric=auc, num_leaves=16; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=2, metric=auc, num_leaves=16; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=2, metric=auc, num_leaves=16; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=2, metric=auc, num_leaves=32; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=2, metric=auc, num_leaves=32; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=2, metric=auc, num_leaves=32; total time=   0.0s
[CV] END boosting_type=gbdt, l

[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=4, metric=auc, num_leaves=8; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=4, metric=auc, num_leaves=16; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=4, metric=auc, num_leaves=16; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=4, metric=auc, num_leaves=16; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=4, metric=auc, num_leaves=16; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=4, metric=auc, num_leaves=32; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=4, metric=auc, num_leaves=32; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=4, metric=auc, num_leaves=32; total time=   0.0s
[CV] END boosting_type=gbdt, learning_rate=0.25, max_depth=4, metric=auc, num_leaves=32; total time=   0.0s
[CV] END boosting_type=gbdt, 

{'boosting_type': 'dart',
 'learning_rate': 0.1,
 'max_depth': 5,
 'metric': 'binary_logloss',
 'num_leaves': 32}

In [91]:
lgbm_model(xtrain_balanced, ytrain_balanced,Xval_tenfeat, Yval )

Fitting 4 folds for each of 360 candidates, totalling 1440 fits
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=2, metric=binary_logloss, num_leaves=16; total t

[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=8; total time=   0.1s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=16; total time=   0.1s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=16; total time=   0.1s
[CV] END boosting_type=dart, learning_rate=0.05, max_depth=4, metric=binary_logloss, num_leaves=16; total time=   0.1s
[CV] END boosting_type=dart, learning_rate=0.05, max

[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=2, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=

[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_logloss, num_leaves=32; total time=   0.1s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_logloss, num_leaves=32; total time=   0.1s
[CV] END boosting_type=dart, learning_rate=0.1, max_depth=4, metric=binary_logloss, num_leaves=32; total time=   0.1s
[CV] END boosting_type=dart, learning_rate=0.1, max_dept

[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_logloss, num_leaves=16; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_logloss, num_leaves=32; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_error, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_depth=2, metric=binary_error, num_leaves=8; total time=   0.0s
[CV] END boosting_type=dart, learning_rate=0.15, max_d

...

[589]	training's binary_logloss: 0.556257	valid_1's binary_logloss: 0.618066
[590]	training's binary_logloss: 0.556337	valid_1's binary_logloss: 0.617999
[591]	training's binary_logloss: 0.556422	valid_1's binary_logloss: 0.617929
[592]	training's binary_logloss: 0.556513	valid_1's binary_logloss: 0.61785
[593]	training's binary_logloss: 0.556409	valid_1's binary_logloss: 0.618022
[594]	training's binary_logloss: 0.556496	valid_1's binary_logloss: 0.617956
[595]	training's binary_logloss: 0.556577	valid_1's binary_logloss: 0.617891
[596]	training's binary_logloss: 0.556387	valid_1's binary_logloss: 0.61815
[597]	training's binary_logloss: 0.556467	valid_1's binary_logloss: 0.618071
[598]	training's binary_logloss: 0.55656	valid_1's binary_logloss: 0.618012
[599]	training's binary_logloss: 0.556404	valid_1's binary_logloss: 0.618222
[600]	training's binary_logloss: 0.55631	valid_1's binary_logloss: 0.618353
[601]	training's binary_logloss: 0.556196	valid_1's binary_logloss: 0.618205
[60

[704]	training's binary_logloss: 0.552726	valid_1's binary_logloss: 0.619485
[705]	training's binary_logloss: 0.552794	valid_1's binary_logloss: 0.619428
[706]	training's binary_logloss: 0.552848	valid_1's binary_logloss: 0.619366
[707]	training's binary_logloss: 0.552917	valid_1's binary_logloss: 0.619303
[708]	training's binary_logloss: 0.552795	valid_1's binary_logloss: 0.619433
[709]	training's binary_logloss: 0.552654	valid_1's binary_logloss: 0.619565
[710]	training's binary_logloss: 0.552518	valid_1's binary_logloss: 0.619868
[711]	training's binary_logloss: 0.552583	valid_1's binary_logloss: 0.619815
[712]	training's binary_logloss: 0.552647	valid_1's binary_logloss: 0.619742
[713]	training's binary_logloss: 0.552581	valid_1's binary_logloss: 0.61986
[714]	training's binary_logloss: 0.55246	valid_1's binary_logloss: 0.620018
[715]	training's binary_logloss: 0.552536	valid_1's binary_logloss: 0.619945
[716]	training's binary_logloss: 0.552469	valid_1's binary_logloss: 0.62007
[7

[820]	training's binary_logloss: 0.549611	valid_1's binary_logloss: 0.620367
[821]	training's binary_logloss: 0.549361	valid_1's binary_logloss: 0.620377
[822]	training's binary_logloss: 0.549412	valid_1's binary_logloss: 0.620328
[823]	training's binary_logloss: 0.549236	valid_1's binary_logloss: 0.620381
[824]	training's binary_logloss: 0.549152	valid_1's binary_logloss: 0.620484
[825]	training's binary_logloss: 0.549039	valid_1's binary_logloss: 0.620612
[826]	training's binary_logloss: 0.549091	valid_1's binary_logloss: 0.620548
[827]	training's binary_logloss: 0.549143	valid_1's binary_logloss: 0.620497
[828]	training's binary_logloss: 0.548941	valid_1's binary_logloss: 0.62042
[829]	training's binary_logloss: 0.548807	valid_1's binary_logloss: 0.620559
[830]	training's binary_logloss: 0.548679	valid_1's binary_logloss: 0.620681
[831]	training's binary_logloss: 0.548536	valid_1's binary_logloss: 0.620764
[832]	training's binary_logloss: 0.548484	valid_1's binary_logloss: 0.620838


[929]	training's binary_logloss: 0.545937	valid_1's binary_logloss: 0.621404
[930]	training's binary_logloss: 0.545819	valid_1's binary_logloss: 0.621418
[931]	training's binary_logloss: 0.545859	valid_1's binary_logloss: 0.621361
[932]	training's binary_logloss: 0.54591	valid_1's binary_logloss: 0.621301
[933]	training's binary_logloss: 0.545852	valid_1's binary_logloss: 0.621341
[934]	training's binary_logloss: 0.545813	valid_1's binary_logloss: 0.621395
[935]	training's binary_logloss: 0.545764	valid_1's binary_logloss: 0.621471
[936]	training's binary_logloss: 0.545815	valid_1's binary_logloss: 0.621407
[937]	training's binary_logloss: 0.545882	valid_1's binary_logloss: 0.621329
[938]	training's binary_logloss: 0.545803	valid_1's binary_logloss: 0.621442
[939]	training's binary_logloss: 0.545847	valid_1's binary_logloss: 0.621396
[940]	training's binary_logloss: 0.545749	valid_1's binary_logloss: 0.621499
[941]	training's binary_logloss: 0.545789	valid_1's binary_logloss: 0.621444


LGBMClassifier(boosting_type='dart', max_depth=5, metric='binary_logloss',
               num_boost_round=1000, num_leaves=32, num_threads=4,
               objective='binary')